In [83]:
# Import cell
import pathlib
from collections import defaultdict
import json
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt


In [84]:
# if fodler trading_strat_data does not exist, create it
pathlib.Path('trading_strat_data').mkdir(parents=True, exist_ok=True)

## sanity check of the data, do we have indeed 30 stock at all time for the DJIA, etc.

In [85]:
# Path to the parent directory containing year folders
data_dir = pathlib.Path("api-data-signal/")

year_to_companies = {}
company_to_years = defaultdict(set)

# Traverse each "year" folder
for year_folder in data_dir.iterdir():
    if year_folder.is_dir():
        year = year_folder.name
        companies = []
        # Each subfolder inside 'year_folder' is treated as a ticker
        for ticker_folder in year_folder.iterdir():
            if ticker_folder.is_dir():
                companies.append(ticker_folder.name)
                company_to_years[ticker_folder.name].add(year)
        year_to_companies[year] = companies

# Build output lines
output_lines = []
output_lines.append("Yearly structure:")
for year, companies in sorted(year_to_companies.items()):
    output_lines.append(f"{year}: {len(companies)} companies")
    output_lines.append(f"  {sorted(companies)}")

output_lines.append("\nCompany appearances across years:")
for company, years in sorted(company_to_years.items()):
    output_lines.append(f"{company}: appears in {len(years)} year(s)")
    output_lines.append(f"  {sorted(years)}")

# Write to a file in the "trading_strat_data/" folder
with open("trading_strat_data/summary_output.txt", "w", encoding="utf-8") as f:
    for line in output_lines:
        f.write(line + "\n")

print("Summary written to summary_output.txt")


Summary written to summary_output.txt


## create single dataframe containing the sentiment values of interest for each stock, each year.

In [86]:
# Set the path to the folder containing the year folders
data_dir = pathlib.Path("api-data-signal/")

# List to hold rows for the DataFrame
rows = []

# Iterate over each year folder
for year_folder in data_dir.iterdir():
    if year_folder.is_dir():
        year = year_folder.name  # e.g., "2010", "2011", etc.
        # Iterate over each company folder within the year folder
        for company_folder in year_folder.iterdir():
            if company_folder.is_dir():
                company = company_folder.name  # e.g., "AAPL", "AMZN", etc.
                info_file = company_folder / "10_K_info.txt"
                if info_file.exists():
                    try:
                        # Read and parse the JSON content from the file
                        with open(info_file, "r", encoding="utf-8") as f:
                            content = f.read().strip()
                            # If the file is pure JSON, we can load it directly
                            info = json.loads(content)
                    except Exception as e:
                        print(f"Error reading {info_file}: {e}")
                        info = {}
                    
                    # Extract sentiment and other values
                    sentiment = info.get("sentiment_score", {})
                    row = {
                        "Year": year,
                        "Company": company,
                        "sentiment_score_positive": sentiment.get("Positive"),
                        "sentiment_score_negative": sentiment.get("Negative"),
                        "sentiment_score_polarity": sentiment.get("Polarity"),
                        "sentiment_score_subjectivity": sentiment.get("Subjectivity"),
                        "similarity_score": info.get("similarity_score"),
                        "nlp_result": info.get("nlp_result")
                    }
                    rows.append(row)
                else:
                    print(f"File not found: {info_file}")

# Create DataFrame
df = pd.DataFrame(rows)

# Optionally convert 'Year' to numeric and sort the DataFrame
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df.sort_values(by=["Year", "Company"]).reset_index(drop=True)

# Display the DataFrame
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv("trading_strat_data/sentiment_analysis.csv", index=False)
print("DataFrame saved to sentiment_analysis.csv")


   Year Company  sentiment_score_positive  sentiment_score_negative  \
0  2010    AAPL                      66.0                     144.0   
1  2010    AMGN                     106.0                     106.0   
2  2010    AMZN                     110.0                      92.0   
3  2010     AXP                       1.0                       0.0   
4  2010      BA                     183.0                     379.0   

   sentiment_score_polarity  sentiment_score_subjectivity  similarity_score  \
0                 -0.371429                      0.047436          0.962567   
1                  0.000000                      0.045778          0.962118   
2                  0.089109                      0.056933          0.000000   
3                  0.999999                      0.041667          0.000000   
4                 -0.348754                      0.073889          0.982955   

   nlp_result  
0           0  
1           0  
2           0  
3           0  
4           0  
Da

In [87]:
# check the number of 0's in the dataframe df, within each column, and their percentage
zero_counts = df.isin([0]).sum()
zero_percentages = df.isin([0]).mean() * 100
print(pd.concat([zero_counts, zero_percentages], axis=1, keys=['Count', 'Percentage']))


                              Count  Percentage
Year                              0    0.000000
Company                           0    0.000000
sentiment_score_positive         32    7.619048
sentiment_score_negative         81   19.285714
sentiment_score_polarity         37    8.809524
sentiment_score_subjectivity     30    7.142857
similarity_score                  4    0.952381
nlp_result                      420  100.000000


## Retrieve market data of ADJUSTED returns (bc of stock splits, etc), and taking for year i+1, then compute returns timeframe for 1)stocks 2)DJIA as a whole

In [88]:
# Define tickers for stocks (example: 30 DJIA companies) and the DJIA ticker
tickers = [
    "AAPL", "AMGN", "AMZN", "AXP", "BA", "CAT", "CRM", "CSCO", "CVX",
    "DIS", "GS", "HD", "HON", "IBM", "JNJ", "JPM", "KO", "MCD", "MMM",
    "MRK", "MSFT", "NKE", "NVDA", "PG", "SHW", "TRV", "UNH", "V", "VZ", "WMT"
]
djia_ticker = "^DJI"

# Define date range
start_date = "2011-01-01"
end_date = "2024-12-31"

# Download adjusted daily closing data for stocks (adjusted prices)
stocks_data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)["Close"]

# Resample to yearly frequency: Get first and last prices of each year
yearly_first = stocks_data.resample("Y").first()
yearly_last = stocks_data.resample("Y").last()

# Calculate yearly returns as (last / first) - 1
yearly_returns = (yearly_last / yearly_first) - 1

# Download adjusted DJIA data and compute yearly returns similarly
djia_data = yf.download(djia_ticker, start=start_date, end=end_date, auto_adjust=True)["Close"]
djia_yearly_first = djia_data.resample("Y").first()
djia_yearly_last = djia_data.resample("Y").last()
djia_yearly_returns = (djia_yearly_last / djia_yearly_first) - 1

# Create DataFrames to display
print("Yearly Stock Returns:")
print(yearly_returns.head())

print("\nYearly DJIA Returns:")
print(djia_yearly_returns.head())

# Optionally, save the results to CSV files:
yearly_returns.to_csv("trading_strat_data/yearly_stock_returns.csv")
djia_yearly_returns.to_csv("trading_strat_data/yearly_djia_returns.csv")


[*********************100%***********************]  30 of 30 completed
/var/folders/cz/tndvsnfj5pbc0hg4zwfqf21c0000gn/T/ipykernel_21146/183208183.py:17: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_first = stocks_data.resample("Y").first()
/var/folders/cz/tndvsnfj5pbc0hg4zwfqf21c0000gn/T/ipykernel_21146/183208183.py:18: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_last = stocks_data.resample("Y").last()
[*********************100%***********************]  1 of 1 completed

Yearly Stock Returns:
Ticker          AAPL      AMGN      AMZN       AXP        BA       CAT  \
Date                                                                     
2011-12-31  0.228874  0.167906 -0.060363  0.104228  0.131701 -0.019657   
2012-12-31  0.305586  0.370443  0.401274  0.204688  0.039845 -0.020585   
2013-12-31  0.047508  0.303361  0.549843  0.558365  0.806242 -0.008824   
2014-12-31  0.426284  0.402192 -0.220167  0.051461 -0.026749  0.045588   
2015-12-31 -0.020822  0.035653  1.190749 -0.241932  0.140914 -0.232671   

Ticker           CRM      CSCO       CVX       DIS  ...      MSFT       NKE  \
Date                                                ...                       
2011-12-31 -0.257247 -0.107641  0.193773  0.008189  ... -0.047545  0.135907   
2012-12-31  0.661067  0.079512  0.012824  0.319618  ...  0.025976  0.081710   
2013-12-31  0.289712  0.127668  0.168769  0.513566  ...  0.395438  0.537844   
2014-12-31  0.081115  0.294883 -0.064119  0.250651  ...  0.28422


/var/folders/cz/tndvsnfj5pbc0hg4zwfqf21c0000gn/T/ipykernel_21146/183208183.py:25: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  djia_yearly_first = djia_data.resample("Y").first()
/var/folders/cz/tndvsnfj5pbc0hg4zwfqf21c0000gn/T/ipykernel_21146/183208183.py:26: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  djia_yearly_last = djia_data.resample("Y").last()


In [89]:
# Plot the DJIA yearly returns
plt.figure(figsize=(10, 6))
plt.plot(djia_yearly_returns, marker="o", color="b", label="DJIA")
plt.title("Yearly DJIA Returns")
plt.xlabel("Year")
plt.ylabel("Returns")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("trading_strat_data/yearly_djia_returns.png")
plt.close()
plt.show()



# Project Data Overview

This project has generated three key datasets that serve as the foundation for further analysis:

1. **Yearly Returns for Each DJIA Stock**
   - **Description:**  
     A DataFrame where each row represents a year and each column represents one of the 30 DJIA stocks.
   - **Calculation:**  
     Yearly return is calculated using the formula:  
     \[
     \text{Return} = \left(\frac{\text{Last Adjusted Price}}{\text{First Adjusted Price}}\right) - 1
     \]
   - **Purpose:**  
     Provides an overview of the annual performance of each stock.

2. **Yearly Returns for the DJIA Index**
   - **Description:**  
     A separate DataFrame showing the yearly return for the DJIA index as a whole (using the `^DJI` ticker).
   - **Calculation:**  
     Calculated using the same method as the individual stocks.
   - **Purpose:**  
     Offers a benchmark for the overall market performance.

3. **Sentiment Analysis DataFrame**
   - **Description:**  
     A DataFrame that contains sentiment analysis metrics for each company (ticker) for each year.
   - **Metrics Included:**  
     - `sentiment_score_positive`: Number of positive sentiment counts.
     - `sentiment_score_negative`: Number of negative sentiment counts.
     - `sentiment_score_polarity`: Overall polarity of the sentiment.
     - `sentiment_score_subjectivity`: How subjective the sentiment is.
     - `similarity_score`: Similarity metric from NLP analysis.
     - `nlp_result`: Additional NLP-derived result.
   - **Purpose:**  
     Enables analysis of textual sentiment in relation to each company’s performance and trends over the years.


## Merge all dataset in logical way

In [90]:
# --- 1. Load the Sentiment Analysis CSV ---
df_sentiment = pd.read_csv("trading_strat_data/sentiment_analysis.csv")
# Convert Year to numeric and shift by 1 so that sentiment for year i becomes year i+1
df_sentiment["Year"] = pd.to_numeric(df_sentiment["Year"], errors="coerce") + 1

# --- 2. Load and Process the Yearly Stock Returns CSV ---
df_stocks = pd.read_csv("trading_strat_data/yearly_stock_returns.csv", parse_dates=True)
if "Year" not in df_stocks.columns:
    if "Date" in df_stocks.columns:
        df_stocks["Year"] = pd.to_datetime(df_stocks["Date"]).dt.year
    else:
        df_stocks = df_stocks.reset_index()
        if "index" in df_stocks.columns:
            df_stocks = df_stocks.rename(columns={"index": "Date"})
        df_stocks["Year"] = pd.to_datetime(df_stocks["Date"]).dt.year
df_stocks_long = pd.melt(df_stocks, id_vars=["Year"], var_name="Company", value_name="Yearly_Return")

# --- 3. Load and Process the Yearly DJIA Returns CSV ---
df_djia = pd.read_csv("trading_strat_data/yearly_djia_returns.csv", parse_dates=True)
if "Year" not in df_djia.columns:
    if "Date" in df_djia.columns:
        df_djia["Year"] = pd.to_datetime(df_djia["Date"]).dt.year
    else:
        df_djia = df_djia.reset_index()
        if "index" in df_djia.columns:
            df_djia = df_djia.rename(columns={"index": "Date"})
        df_djia["Year"] = pd.to_datetime(df_djia["Date"]).dt.year
djia_return_col = [col for col in df_djia.columns if col not in ["Year", "Date"]][0]
df_djia = df_djia.rename(columns={djia_return_col: "DJIA_Return"})

# --- 4. Merge the DataFrames ---
df_merge = pd.merge(df_sentiment, df_stocks_long, on=["Year", "Company"], how="inner")
df_final = pd.merge(df_merge, df_djia[["Year", "DJIA_Return"]], on="Year", how="left")

# --- 5. Output ---
print(df_final.head())
df_final.to_csv("trading_strat_data/merged_data.csv", index=False)
print("Merged data saved to trading_strat_data/merged_data.csv")


   Year Company  sentiment_score_positive  sentiment_score_negative  \
0  2011    AAPL                      66.0                     144.0   
1  2011    AMGN                     106.0                     106.0   
2  2011    AMZN                     110.0                      92.0   
3  2011     AXP                       1.0                       0.0   
4  2011      BA                     183.0                     379.0   

   sentiment_score_polarity  sentiment_score_subjectivity  similarity_score  \
0                 -0.371429                      0.047436          0.962567   
1                  0.000000                      0.045778          0.962118   
2                  0.089109                      0.056933          0.000000   
3                  0.999999                      0.041667          0.000000   
4                 -0.348754                      0.073889          0.982955   

   nlp_result Yearly_Return  DJIA_Return  
0           0      0.228874     0.046853  
1           

In [74]:
# import pandas as pd
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_absolute_error, r2_score

# # Load the merged data (merged_data.csv already has sentiment and Yearly_Return)
# df = pd.read_csv("trading_strat_data/merged_data.csv")

# # Sort by Company and Year
# df = df.sort_values(by=["Company", "Year"])

# # Create target: shift Yearly_Return by -1 so that sentiment for year i predicts return in year i+1
# df["Target_Return"] = df.groupby("Company")["Yearly_Return"].shift(-1)
# df = df.dropna(subset=["Target_Return"])  # Drop rows where target is missing

# # Define features (sentiment metrics) and target variable
# features = [
#     "sentiment_score_positive", 
#     "sentiment_score_negative", 
#     "sentiment_score_polarity", 
#     "sentiment_score_subjectivity", 
#     "similarity_score"
# ]
# target = "Target_Return"

# # Ensure Year is numeric
# df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

# # Split data chronologically:
# # Training: rows with Year ≤ 2018 (sentiment from i predicts return for i+1, where i is ≤2018)
# # Testing: rows with Year > 2018
# train = df[df["Year"] <= 2018]
# test = df[df["Year"] > 2018]

# X_train = train[features]
# y_train = train[target]
# X_test = test[features]
# y_test = test[target]

# # Train a linear regression model
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Predict on test set
# y_pred = model.predict(X_test)

# # Evaluate model
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print("Test MAE:", mae)
# print("Test R2 Score:", r2)

# # Display sample predictions with actual values
# results = test.copy()
# results["Predicted_Return"] = y_pred
# print(results[["Year", "Company", "Yearly_Return", "Target_Return", "Predicted_Return"]].head())


### take all column from ´company´ to ´similarity_score´ as variables, regress targetting returns

In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Load the merged data (assume it's saved as 'merged_data.csv')
df = pd.read_csv("trading_strat_data/merged_data.csv")

# Keep only the relevant columns (ignoring DJIA_Return)
columns_to_keep = [
    "Year", "Company", "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score",
    "Yearly_Return"
]
df = df[columns_to_keep]

# One-hot encode the 'Company' column
encoder = OneHotEncoder(sparse_output=False, drop='first')
company_encoded = encoder.fit_transform(df[['Company']])
company_encoded_df = pd.DataFrame(company_encoded, columns=encoder.get_feature_names_out(['Company']))
df = pd.concat([df.reset_index(drop=True), company_encoded_df.reset_index(drop=True)], axis=1)
df.drop(columns=["Company"], inplace=True)

# Define the target and features
target = "Yearly_Return"
features = [
    "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score"
] + list(company_encoded_df.columns)

# Sort data by Year to maintain time order
df.sort_values("Year", inplace=True)

# Perform a time-based split: training on 2011-2020, testing on 2021-2024
train_data = df[df["Year"] <= 2020]
test_data  = df[df["Year"] > 2020]

X_train = train_data[features]
y_train = train_data[target]
X_test  = test_data[features]
y_test  = test_data[target]

# Initialize and train the linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lr_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Mean Squared Error on test set: 0.1338316475881662


### take all column from ´sentiment_score_positive´ to ´similarity_score´ as variables, regress targetting returns

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the merged data (assume it's saved as 'merged_data.csv')
df = pd.read_csv("trading_strat_data/merged_data.csv")

# Keep only the relevant columns (ignoring DJIA_Return and Company)
columns_to_keep = [
    "Year", "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score",
    "Yearly_Return"
]
df = df[columns_to_keep]

# Define the target and features (Company is not used)
target = "Yearly_Return"
features = [
    "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score"
]

# Sort data by Year to maintain time order
df.sort_values("Year", inplace=True)

# Perform a time-based split: training on 2011-2020, testing on 2021-2024
train_data = df[df["Year"] <= 2020]
test_data  = df[df["Year"] > 2020]

X_train = train_data[features]
y_train = train_data[target]
X_test  = test_data[features]
y_test  = test_data[target]

# Initialize and train the linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lr_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Mean Squared Error on test set: 0.15925298615904512


In [96]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Load the merged data (assume it's saved as 'merged_data.csv')
df = pd.read_csv("trading_strat_data/merged_data.csv")

# Keep only the relevant columns (ignoring DJIA_Return and Company)
columns_to_keep = [
    "Year", "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score",
    "Yearly_Return"
]
df = df[columns_to_keep]

# Sort data by Year to maintain time order
df.sort_values("Year", inplace=True)

# Define the target and features
target = "Yearly_Return"
features = [
    "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score"
]

# Perform a time-based split: training on 2011-2020, testing on 2021-2024
train_data = df[df["Year"] <= 2020]
test_data  = df[df["Year"] > 2020]

X_train = train_data[features].values
y_train = train_data[target].values
X_test  = test_data[features].values
y_test  = test_data[target].values

# Standardize features for better neural network training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build a simple neural network model
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))  # Output layer for regression (no activation)

model.compile(optimizer='adam', loss='mse')

# Setup early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train, 
    epochs=100, 
    batch_size=8, 
    validation_split=0.2, 
    callbacks=[early_stopping],
    verbose=1
)

# Predict on the test set
y_pred = model.predict(X_test).flatten()

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Epoch 1/100


/Users/alexandrebrun/miniconda3/envs/data_analysis_finance/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0963 - val_loss: 0.1059
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0807 - val_loss: 0.0885
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0951 - val_loss: 0.0750
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0807 - val_loss: 0.0781
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0581 - val_loss: 0.0702
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0804 - val_loss: 0.0728
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0638 - val_loss: 0.0745
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0775 - val_loss: 0.0703
Epoch 9/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0408 - val_loss: 0.0707
Epoch 10/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0530 - val_loss: 0.0701
Epoch 11/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0625 - val_loss: 0.0712
Epoch 12/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0513 - val_lo

## + advanced

In [100]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Load and preprocess the data
df = pd.read_csv("trading_strat_data/merged_data.csv")
columns_to_keep = [
    "Year", "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score",
    "Yearly_Return"
]
df = df[columns_to_keep]
df.sort_values("Year", inplace=True)

target = "Yearly_Return"
features = [
    "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score"
]

train_data = df[df["Year"] <= 2020]
test_data  = df[df["Year"] > 2020]

X_train = train_data[features].values
y_train = train_data[target].values
X_test  = test_data[features].values
y_test  = test_data[target].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build an enhanced neural network model
model = Sequential()
# First layer with L2 regularization, BatchNormalization, and Dropout
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Second layer
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Third layer (optional additional layer)
model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())

# Output layer
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

# Additional callback to reduce learning rate when a plateau is reached
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train, 
    epochs=100, 
    batch_size=8, 
    validation_split=0.2, 
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Predict and evaluate
y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Epoch 1/100


/Users/alexandrebrun/miniconda3/envs/data_analysis_finance/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2.4495 - val_loss: 0.2570 - learning_rate: 0.0010
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8129 - val_loss: 0.2612 - learning_rate: 0.0010
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2362 - val_loss: 0.2524 - learning_rate: 0.0010
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8623 - val_loss: 0.2424 - learning_rate: 0.0010
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7739 - val_loss: 0.2293 - learning_rate: 0.0010
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5757 - val_loss: 0.2381 - learning_rate: 0.0010
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5350 - val_loss: 0.2395 - learning_rate: 0.0010
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3822 - val_loss: 0.2393 - learning_rate: 0.0010
Epoch 9/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4262 - val_loss: 0.2357 - learning_rate: 0.0010
Epoch 10/100
30/30 ━━━━━━━━━━━━

### XGBOOST

In [98]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Load the merged data (assume it's saved as 'trading_strat_data/merged_data.csv')
df = pd.read_csv("trading_strat_data/merged_data.csv")

# Keep only the relevant columns (ignoring DJIA_Return and Company)
columns_to_keep = [
    "Year", "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score",
    "Yearly_Return"
]
df = df[columns_to_keep]

# Define the target and features (Company is not used)
target = "Yearly_Return"
features = [
    "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score"
]

# Sort data by Year to maintain time order
df.sort_values("Year", inplace=True)

# Perform a time-based split: training on 2011-2020, testing on 2021-2024
train_data = df[df["Year"] <= 2020]
test_data  = df[df["Year"] > 2020]

X_train = train_data[features]
y_train = train_data[target]
X_test  = test_data[features]
y_test  = test_data[target]

# Initialize and train the XGBoost regressor
xgb_model = XGBRegressor(
    objective='reg:squarederror',  # Using squared error for regression
    n_estimators=100,               # Number of trees
    learning_rate=0.1,              # Learning rate
    max_depth=3,                    # Maximum depth of trees
    random_state=42
)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Mean Squared Error on test set: 0.15021198841130426


## back to reg (ridge)

In [99]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Load the merged data (assume it's saved as 'trading_strat_data/merged_data.csv')
df = pd.read_csv("trading_strat_data/merged_data.csv")

# Keep only the relevant columns (ignoring DJIA_Return and Company)
columns_to_keep = [
    "Year", "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score",
    "Yearly_Return"
]
df = df[columns_to_keep]

# Define the target and features (Company is not used)
target = "Yearly_Return"
features = [
    "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score"
]

# Sort data by Year to maintain time order
df.sort_values("Year", inplace=True)

# Perform a time-based split: training on 2011-2020, testing on 2021-2024
train_data = df[df["Year"] <= 2020]
test_data  = df[df["Year"] > 2020]

X_train = train_data[features]
y_train = train_data[target]
X_test  = test_data[features]
y_test  = test_data[target]

# Initialize and train the Ridge regression model
# Adjust alpha as necessary; a common starting point is alpha=1.0.
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ridge_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Mean Squared Error on test set: 0.15234047547600227


## I) 

-try to pool all together/ not pool and keep company per company
(training assuming that all synthax of files "10_K_info.txt" are in similar style: hence we don't trian company per company?)

-try algo reg, NeuralNet, etc?

-which inputs

## II)
Trading strat and backtest?